In [1]:
import os
import datetime
from datetime import timedelta
import time
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime

time_run = time.time()

In [2]:
xls = pd.ExcelFile("../Input file/Costing Examples_NEW.xlsx")
time_df = pd.read_excel(xls, 'Check')

holiday_df = xls.parse("Holiday")
holiday_list = [("" if m>9 else "0") + str(m) + "/" + ("" if d>9 else "0") + str(d) for d, m in zip(holiday_df["Day"], holiday_df["Month"])]

base_rate = 20
holiday_rate = 2.5

rate1 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate2 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate3 = np.array([[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.7,1.7,1.7,1.7,2.2],[2.2,2.2,2.2,2.2,2.2]])   
rate_data = {"Full Time": rate1, "Part Time": rate2, "Casual": rate3}

In [3]:
def date_to_str(date):
    date = str(date).split("-")
    return date[1] + "/" + date[2] 

def day_val(day):
    val = {"Monday":0,"Tuesday":1,"Wednesday":2,'Thursday':3,"Friday":4,"Saturday":5,"Sunday":6}
    return val[day]

def cal_base_rate(_type, day, date, start_time, end_time, rate_data):
    
    shift = 2;      
    ord_rate = 0
    ot1_rate = 0
    ot2_rate = 0
    if date_to_str(date) in holiday_list:
        ord_rate = holiday_rate
        ot1_rate = holiday_rate
        ot2_rate = holiday_rate
    else:        
        if _type == "Full Time":
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        elif _type == "Part Time":
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0

        else:
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        ord_rate = rate_data[_type][day_val(day)][shift]
        ot1_rate = rate_data[_type][day_val(day)][3]
        ot2_rate = rate_data[_type][day_val(day)][4]
    return (ord_rate,ot1_rate,ot2_rate)

In [4]:

data= []
list_pay = []
for row in time_df.iterrows():    
    _id = row[1]["Contact Name"] 
    
    start_period = row[1]['Process period start date'].date()
    end_period = row[1]['Process period end date'].date()
    
    start = row[1]["Start date"]
    end   = row[1]["End date"]   
    start_time = start.hour + start.minute/60
    end_time = end.hour + end.minute/60
        
    _type = row[1]["Employment Type"]
    
    start_date = start.date() 
    end_date   = end.date()
    
    delta_day = int((start_date- end_date).days)
    
    start_day  = start.strftime('%A')
    end_day    = end.strftime('%A')  
    if ((start_date != end_date) and ( ((end_time >=1) and (start_day in ["Friday","Saturday","Sunday"])) or ( date_to_str(start_date) in holiday_list or date_to_str(end_date) in holiday_list ) )):
        
        rate1 = cal_base_rate(_type, start_day, start_date, start_time, 24, rate_data)[0]
        rate2 = cal_base_rate(_type, end_day, end_date, 0, end_time, rate_data)[0]
        ot1_rate1 = cal_base_rate(_type, start_day, start_date, start_time, 24, rate_data)[1]
        ot1_rate2 = cal_base_rate(_type, end_day, end_date, 0, end_time, rate_data)[1]
        ot2_rate1 = cal_base_rate(_type, start_day, start_date, start_time, 24, rate_data)[2]
        ot2_rate2 = cal_base_rate(_type, end_day, end_date, 0, end_time, rate_data)[2]
        
        delta_time1 = 24 - start_time   
        delta_time2 = end_time      
        if delta_time1 <3 :
            delta_time1=3
        if delta_time2<3:
            delta_time2=3
        start_date1 = start_date   
        end_date1= end_date
        start_date2 = end_date
        end_date2= end_date
        
        start1= start
        end2=end
        end1= end.replace(hour=0,minute=0)
        start2=end1
        
        start_day1=start1.strftime('%A')
        end_day1=end1.strftime('%A')
        start_day2=start2.strftime('%A')
        end_day2=end2.strftime('%A')
        
        
        
        data.append((_id,start_period,start_date1,start1, end1, start_time, 24 ,start_day1, delta_time1, rate1,ot1_rate1,ot2_rate1))
        data.append((_id,start_period,start_date2, start2, end2,  0, end_time, start_day2,delta_time2, rate2,ot1_rate2,ot2_rate2))  
     
    else: 
        delta_time = (end-start).total_seconds() / 3600.0
        if delta_time <3:
            delta_time = 3
        rate = cal_base_rate(_type, start_day, start_date, start_time, end_time, rate_data)[0]
        ot1_rate = cal_base_rate(_type, start_day, start_date, start_time, end_time, rate_data)[1]
        ot2_rate = cal_base_rate(_type, start_day, start_date, start_time, end_time, rate_data)[2]
        
        data.append((_id,start_period,start_date,start,end, start_time, end_time,start_day, delta_time, rate,ot1_rate,ot2_rate))
            
            
file = pd.DataFrame(data,columns=['ID',"Start Period",'Start Date','Start','End', "Start Hour", "End Hour","Day",'Hour','Rate',"Rate OT1","Rate OT2"])        

file
    

,ID,Start Period,Start Date,Start,End,Start Hour,End Hour,Day,Hour,Rate,Rate OT1,Rate OT2
0,Aaron Edwards,2021-03-22,2021-03-26,2021-03-26 22:00:00,2021-03-27 00:30:00,22.0,0.5,Friday,3.0,1.35,1.7,2.2
1,Aaron Edwards,2021-03-22,2021-03-26,2021-03-26 22:00:00,2021-03-27 00:00:00,22.0,24.0,Friday,3.0,1.45,1.7,2.2
2,Aaron Edwards,2021-03-22,2021-03-27,2021-03-27 00:00:00,2021-03-27 02:00:00,0.0,2.0,Saturday,3.0,1.70,1.7,2.2
3,Quynh,2021-03-22,2021-03-27,2021-03-27 22:00:00,2021-03-28 00:00:00,22.0,24.0,Saturday,3.0,1.70,1.7,2.2
4,Quynh,2021-03-22,2021-03-28,2021-03-28 00:00:00,2021-03-28 07:00:00,0.0,7.0,Sunday,7.0,2.20,2.2,2.2
5,Quynh,2021-03-22,2021-03-26,2021-03-26 00:00:00,2021-03-27 00:00:00,0.0,0.0,Friday,24.0,1.25,1.5,2.0
6,Aaron Edwards,2021-03-22,2021-03-25,2021-03-25 06:00:00,2021-03-25 19:30:00,6.0,19.5,Thursday,13.5,1.25,1.5,2.0
7,Quynh,2021-03-22,2021-03-26,2021-03-26 06:00:00,2021-03-26 13:30:00,6.0,13.5,Friday,7.5,1.25,1.5,2.0
8,Aaron Edwards,2021-04-22,2021-04-25,2021-04-25 06:00:00,2021-04-25 13:30:00,6.0,13.5,Sunday,7.5,2.50,2.5,2.5
9,Quynh,2021-04-22,2021-04-24,2021-04-24 20:00:00,2021-04-25 00:00:00,20.0,24.0,Saturday,4.0,1.50,1.5,2.0


In [14]:
file = file.sort_values(['ID',"Start Period"])
file

,ID,Start Period,Start Date,Start,End,Start Hour,End Hour,Day,Hour,Rate,Rate OT1,Rate OT2
0,Aaron Edwards,2021-03-22,2021-03-26,2021-03-26 22:00:00,2021-03-27 00:30:00,22.0,0.5,Friday,3.0,1.35,1.7,2.2
1,Aaron Edwards,2021-03-22,2021-03-26,2021-03-26 22:00:00,2021-03-27 00:00:00,22.0,24.0,Friday,3.0,1.45,1.7,2.2
2,Aaron Edwards,2021-03-22,2021-03-27,2021-03-27 00:00:00,2021-03-27 02:00:00,0.0,2.0,Saturday,3.0,1.70,1.7,2.2
6,Aaron Edwards,2021-03-22,2021-03-25,2021-03-25 06:00:00,2021-03-25 19:30:00,6.0,19.5,Thursday,13.5,1.25,1.5,2.0
8,Aaron Edwards,2021-04-22,2021-04-25,2021-04-25 06:00:00,2021-04-25 13:30:00,6.0,13.5,Sunday,7.5,2.50,2.5,2.5
3,Quynh,2021-03-22,2021-03-27,2021-03-27 22:00:00,2021-03-28 00:00:00,22.0,24.0,Saturday,3.0,1.70,1.7,2.2
4,Quynh,2021-03-22,2021-03-28,2021-03-28 00:00:00,2021-03-28 07:00:00,0.0,7.0,Sunday,7.0,2.20,2.2,2.2
5,Quynh,2021-03-22,2021-03-26,2021-03-26 00:00:00,2021-03-27 00:00:00,0.0,0.0,Friday,24.0,1.25,1.5,2.0
7,Quynh,2021-03-22,2021-03-26,2021-03-26 06:00:00,2021-03-26 13:30:00,6.0,13.5,Friday,7.5,1.25,1.5,2.0
9,Quynh,2021-04-22,2021-04-24,2021-04-24 20:00:00,2021-04-25 00:00:00,20.0,24.0,Saturday,4.0,1.50,1.5,2.0


In [15]:
period_pay = {}
for row in file.iterrows():
    _id = row[1]["ID"]
    _period = row[1]["Start Period"] 
    _key_period = _id + " " + str(_period)
    _day = row[1]["Start Date"]
    _hour = row[1]["Hour"]
    _start = row[1]["Start Hour"]
    _end = row[1]["End Hour"]
    _rate = row[1]["Rate"]
    _ot1_rate = row[1]["Rate OT1"]
    _ot2_rate = row[1]["Rate OT2"]
    
    
    value = 0
    total_hour = 0
    pay = base_rate*(min([8,_hour])*_rate + max([0,min([2,_hour - 8])])*_ot1_rate + max([0,min([_hour - 10])])*_ot2_rate)
#     print(pay)
    if not _key_period in period_pay:
        print(_key_period)
        total_hour = _hour
        total_hour += total_hour
        print(total_hour)
        period_pay[_key_period] = value
    else:
        total_hour += total_hour
        

Aaron Edwards 2021-03-22
6.0
Aaron Edwards 2021-04-22
15.0
Quynh 2021-03-22
6.0
Quynh 2021-04-22
8.0
